In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import cPickle as pickle
import multiprocessing
import scipy.special
import numpy as np
import numpy.fft as fft
import time
import os
import sys
sys.path.insert(1,'/Users/zyzdiana/GitHub/AC297r-Volume-Registration/code')

In [3]:
from conversion import rotation_matrix_fromq,rotation_matrix_from_params,rotation_matrix_to_q,angles_from_q
from utils import to_radian,res_to_rad,ax_to_w,rep_to_angle

In [19]:
# Get All Axes of Rotations
path = '/Users/zyzdiana/Dropbox/THESIS/July_26_2016_navs/'
resolutions = ['6_4mm', '8mm', '10mm']
ranges = ['0_5_to_2_5','3_0_to_5_0','2_5_to_12_5','15_to_25']
files = os.listdir(path)
axes_dict = {}
for f in files[1:]:
    rot_ax = f.split('_')[2]
    trans_ax = f.split('_')[10]
    if not axes_dict.has_key(rot_ax):
        axes_dict[rot_ax] = trans_ax
print axes_dict
keys = axes_dict.keys()

rot_axes = ['xz', 'y', 'yz', 'xy', 'x', 'z']
ax_to_idx = {}
for rot_ax in rot_axes:
    ax_to_idx[rot_ax] = rot_axes.index(rot_ax)
print ax_to_idx

{'xz': 'xy', 'y': 'z', 'yz': 'xy', 'xy': 'xy', 'x': 'z', 'z': 'z'}
{'xy': 3, 'xz': 0, 'yz': 2, 'y': 1, 'x': 4, 'z': 5}


In [5]:
def Max_displacement(dRM, R_axis, dt):
    rad = 100
    trans_R = rad*np.sqrt(3-np.trace(dRM))
    E_max = np.sqrt(trans_R**2 + 2*trans_R*np.linalg.norm(dt-(dt.T.dot(R_axis))*R_axis) + (dt.T).dot(dt))
    return trans_R, E_max

In [6]:
def RMS_Rotation(dRM):
    return np.arccos((np.trace(dRM)-1)/2.)*180/np.pi

def RMS_Translation(dt):
    t = dt
    return np.sqrt((t.T).dot(t))
def RMS_Translation_R(dRM):
    rad = 100
    A = dRM - np.eye(3)
    return np.sqrt(0.2*rad**2*np.trace((A.T).dot(A)))

def RMS(dRM, dt, res_f):
    rad = 100
    A = dRM - np.eye(3)
    return np.sqrt(0.2*rad**2*np.trace((A.T).dot(A))+ (dt.T).dot(dt))
    
from conversion import rotation_matrix_fromq, rotation_matrix_from_params
def get_true_params(rot_angle,rot_ax):
    wy,wx,wz = ax_to_w(rot_ax)
    wy_t,wx_t,wz_t = ax_to_w(axes_dict[rot_ax])
    true_RM = rotation_matrix_fromq(rot_angle[0],wx,wy,wz)
    trans_y, trans_x, trans_z = wy_t*rot_angle[1],wx_t*rot_angle[1], wz_t*rot_angle[1]
    return [trans_x, trans_y, trans_z], true_RM

def get_params(params, res):
    RM = rotation_matrix_from_params(-params[3:])
    return  params[:3]*res, RM

In [26]:
keys = ['xz', 'y', 'yz', 'xy', 'x', 'z']

def compute_RMS_from_C_output(output_path, output_filename):
    results = np.loadtxt(os.path.join(output_path,output_filename))
    splits = output_filename.split('_')
    res = splits[0][:-2]
    res_f = float('.'.join(res.split('_')))
    interp = splits[1]
    rot_ax = splits[2]
    trans_ax = splits[10]
    rang = '_'.join(splits[4:9])
    
    true_params = []
    Max_dis = []
    Max_dis_R = []
    RMS_ls = []
    RMS_rotation_ls = []
    RMS_translation_ls = []
    RMS_trans_R = []
    counter_ls = []    
    
    ref = rep_to_angle(0,rang)
    
    idx = 0
    for rep in xrange(1,36):
        rot_angle = rep_to_angle(rep,rang)
        true_params.append(rot_angle)
        true_t, true_RM = get_true_params(rot_angle,rot_ax)
        rad = res_to_rad(res)
        
        for algo in xrange(8):
            counter = results[idx,1]
            Ps = results[idx,2:]
            R_axis = Ps[3:]/np.linalg.norm(Ps[3:])
            t , RM = get_params(Ps, res_f)
            dt = true_t + t
            dR = RM.dot(true_RM.T)
            max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
            Max_dis.append(E_Max)
            Max_dis_R.append(max_trans_R)
            RMS_ls.append(RMS(dR, dt, res_f))
            
            if(RMS(dR, dt, res_f) > 1):
                print idx, rot_ax, rang, rep
            if(RMS(dR, dt, res_f) > E_Max):
                print rot_ax, rang, rep, rot_angle
                print RMS(dR, dt, res_f), E_Max

            RMS_rotation_ls.append(RMS_Rotation(dR))
            RMS_translation_ls.append(RMS_Translation(dt))
            RMS_trans_R.append(RMS_Translation_R(dR))
            counter_ls.append(counter)
            idx += 1
    return [true_params, 
            RMS_ls, RMS_rotation_ls, RMS_translation_ls, 
            Max_dis, Max_dis_R, RMS_trans_R, counter_ls]  


def compute_RMS(output_path, interp, algo, axes_dict):
    true_params = []
    Max_dis_6_4 = []
    Max_dis_R_6_4 = []
    RMS_ls_6_4 = []
    RMS_rotation_ls_6_4 = []
    RMS_translation_ls_6_4 = []
    RMS_trans_R_6_4 = []
    counter_6_4 = []
    
    Max_dis_8 = []
    Max_dis_R_8 = []
    RMS_ls_8 = []
    RMS_rotation_ls_8 = []
    RMS_translation_ls_8 = []
    RMS_trans_R_8 = []
    counter_8 = []
    
    Max_dis_10 = []
    Max_dis_R_10 = []
    RMS_ls_10 = []
    RMS_rotation_ls_10 = []
    RMS_translation_ls_10 = []
    RMS_trans_R_10 = []
    counter_10 = []
    for ix, rot_ax in enumerate(keys):
        for rang in ranges:
            ref = rep_to_angle(0,rang)
            for rep in xrange(1,36):
                rot_angle = rep_to_angle(rep,rang)
                true_params.append(rot_angle)
                true_t, true_RM = get_true_params(rot_angle,rot_ax)

                res = '6_4mm'
                res_f =  6.4
                rad = res_to_rad(res)
                output_file = '%s_%s_%s_rot_%s_deg_%s_trans.txt' % (res, interp, rot_ax, rang, axes_dict[rot_ax])
                results = np.loadtxt(os.path.join(output_path,output_file))
                #for algo in xrange(8):
                idx = algo+(rep-1)*8
                counter = results[idx,1]
                Ps = results[idx,2:]
                R_axis = Ps[3:]/np.linalg.norm(Ps[3:])
                t , RM = get_params(Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_6_4.append(E_Max)
                Max_dis_R_6_4.append(max_trans_R)
                RMS_ls_6_4.append(RMS(dR, dt, res_f))
                if(RMS(dR, dt, res_f) > 1):
                    print ix, rot_ax, rang, rep
                if(RMS(dR, dt, res_f) > E_Max):
                    print rot_ax, rang, rep, rot_angle
                    print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_6_4.append(RMS_Rotation(dR))
                RMS_translation_ls_6_4.append(RMS_Translation(dt))
                RMS_trans_R_6_4.append(RMS_Translation_R(dR))
                counter_6_4.append(counter)

                res = '8mm'
                res_f =  8.0
                rad = res_to_rad(res)
                output_file = '%s_%s_%s_rot_%s_deg_%s_trans.txt' % (res, interp, rot_ax, rang, axes_dict[rot_ax])
                results = np.loadtxt(os.path.join(output_path,output_file))
                #for algo in xrange(8):
                idx = algo+(rep-1)*8
                counter = results[idx,1]
                Ps = results[idx,2:]
                R_axis = Ps[3:]/np.linalg.norm(Ps[3:])
                t , RM = get_params(Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_8.append(E_Max)
                Max_dis_R_8.append(max_trans_R)
                RMS_ls_8.append(RMS(dR, dt, res_f))
                if(RMS(dR, dt, res_f) > 1):
                    print ix, rot_ax, rang, rep
                if(RMS(dR, dt, res_f) > E_Max):
                    print rot_ax, rang, rep, rot_angle
                    print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_8.append(RMS_Rotation(dR))
                RMS_translation_ls_8.append(RMS_Translation(dt))
                RMS_trans_R_8.append(RMS_Translation_R(dR))
                counter_8.append(counter)

                res = '10mm'
                res_f =  10.0
                rad = res_to_rad(res)
                output_file = '%s_%s_%s_rot_%s_deg_%s_trans.txt' % (res, interp, rot_ax, rang, axes_dict[rot_ax])   
                results = np.loadtxt(os.path.join(output_path,output_file))
                #for algo in xrange(8):
                idx = algo+(rep-1)*8        
                counter = results[idx,1]
                Ps = results[idx,2:]
                R_axis = Ps[3:]/np.linalg.norm(Ps[3:])
                t , RM = get_params(Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_10.append(E_Max)
                Max_dis_R_10.append(max_trans_R)
                RMS_ls_10.append(RMS(dR, dt, res_f))
                if(RMS(dR, dt, res_f) > 1):
                    print ix, rot_ax, rang, rep
                if(RMS(dR, dt, res_f) > E_Max):
                    print rot_ax, rang, rep, rot_angle
                    print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_10.append(RMS_Rotation(dR))
                RMS_translation_ls_10.append(RMS_Translation(dt))
                RMS_trans_R_10.append(RMS_Translation_R(dR))
                counter_10.append(counter)
                
    return [true_params,
            RMS_ls_6_4,RMS_ls_8,RMS_ls_10,
            RMS_rotation_ls_6_4,RMS_rotation_ls_8,RMS_rotation_ls_10, 
            RMS_translation_ls_6_4,RMS_translation_ls_8,RMS_translation_ls_10,
            Max_dis_6_4,Max_dis_8,Max_dis_10,
            Max_dis_R_6_4,Max_dis_R_8,Max_dis_R_10,
            RMS_trans_R_6_4, RMS_trans_R_8, RMS_trans_R_10,
            counter_6_4, counter_8, counter_10]

## Get all Bspline results for every subject

In [18]:
interp = 'bspline'
for ix, rot_ax in enumerate(keys):
    for rang in ranges[:2]:
        ref = rep_to_angle(0,rang)
        for rep in xrange(1,36):
            for res in resolutions:
                output_file = '%s_%s_%s_rot_%s_deg_%s_trans.txt' % (res, interp, rot_ax, rang, axes_dict[rot_ax])
                #print output_file

In [ ]:
output_path = '/Users/zyzdiana/Desktop/C_output/'
all_algorithms = []
for algo in xrange(8):
    data_ls = []
    for interp in ['bspline']:
        for subj in ['oct13','june8','july15','july26']:
            data_ls.append(compute_RMS(output_path, interp, algo, axes_dict))
    all_algorithms.append(data_ls)

# Get all data

In [ ]:
output_path = '/Users/zyzdiana/Desktop/'
all_algorithms = []
for algo in xrange(8):
    data_ls = []
    for interp in ['bspline','tricubic','trilinear']:
        for subj in ['oct13','june8','july15','july26']:
            data_ls.append(compute_RMS(output_path, interp, algo, axes_dict))
    all_algorithms.append(data_ls)